In [99]:
from torch.utils.data import Dataset, DataLoader


class TabularDataset(Dataset):
    def __init__(self, data, cat_cols=None, output_col=None):
        """
        Characterizes a Dataset for PyTorch

        Parameters
        ----------

        data: pandas data frame
          The data frame object for the input data. It must
          contain all the continuous, categorical and the
          output columns to be used.

        cat_cols: List of strings
          The names of the categorical columns in the data.
          These columns will be passed through the embedding
          layers in the model. These columns must be
          label encoded beforehand. 

        output_col: string
          The name of the output variable column in the data
          provided.
        """

        self.n = data.shape[0]

        if output_col:
            self.y = data[output_col].astype(np.float32).values.reshape(-1, 1)
        else:
            self.y =  np.zeros((self.n, 1))

        self.cat_cols = cat_cols if cat_cols else []
        self.cont_cols = [col for col in data.columns
                          if col not in self.cat_cols + [output_col]]

        if self.cont_cols:
            self.cont_X = data[self.cont_cols].astype(np.float32).values
        else:
            self.cont_X = np.zeros((self.n, 1))

        if self.cat_cols:
            self.cat_X = data[cat_cols].astype(np.int64).values
        else:
            self.cat_X =  np.zeros((self.n, 1))

    def __len__(self):
        """
        Denotes the total number of samples.
        """
        return self.n

    def __getitem__(self, idx):
        """
        Generates one sample of data.
        """
        return [self.y[idx], self.cont_X[idx], self.cat_X[idx]]
    

In [100]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [101]:
class FeedForwardNN(nn.Module):

    def __init__(self, emb_dims, no_of_cont, lin_layer_sizes,
               output_size, emb_dropout, lin_layer_dropouts):

        """
        Parameters
        ----------

        emb_dims: List of two element tuples
          This list will contain a two element tuple for each
          categorical feature. The first element of a tuple will
          denote the number of unique values of the categorical
          feature. The second element will denote the embedding
          dimension to be used for that feature.

        no_of_cont: Integer
          The number of continuous features in the data.

        lin_layer_sizes: List of integers.
          The size of each linear layer. The length will be equal
          to the total number
          of linear layers in the network.

        output_size: Integer
          The size of the final output.

        emb_dropout: Float
          The dropout to be used after the embedding layers.

        lin_layer_dropouts: List of floats
          The dropouts to be used after each linear layer.
        """

        super().__init__()

        # Embedding layers
        self.emb_layers = nn.ModuleList([nn.Embedding(x, y)
                                         for x, y in emb_dims])

        no_of_embs = sum([y for x, y in emb_dims])
        self.no_of_embs = no_of_embs
        self.no_of_cont = no_of_cont

        # Linear Layers
        first_lin_layer = nn.Linear(self.no_of_embs + self.no_of_cont,
                                    lin_layer_sizes[0])

        self.lin_layers =\
         nn.ModuleList([first_lin_layer] +\
              [nn.Linear(lin_layer_sizes[i], lin_layer_sizes[i + 1])
               for i in range(len(lin_layer_sizes) - 1)])

        for lin_layer in self.lin_layers:
            nn.init.kaiming_normal_(lin_layer.weight.data)

        # Output Layer
        self.output_layer = nn.Linear(lin_layer_sizes[-1],
                                      output_size)
        nn.init.kaiming_normal_(self.output_layer.weight.data)

        # Batch Norm Layers
        self.first_bn_layer = nn.BatchNorm1d(self.no_of_cont)
        self.bn_layers = nn.ModuleList([nn.BatchNorm1d(size)
                                        for size in lin_layer_sizes])

        # Dropout Layers
        self.emb_dropout_layer = nn.Dropout(emb_dropout)
        self.droput_layers = nn.ModuleList([nn.Dropout(size)
                                      for size in lin_layer_dropouts])

    def forward(self, cont_data, cat_data):

        if self.no_of_embs != 0:
            x = [emb_layer(cat_data[:, i])
               for i,emb_layer in enumerate(self.emb_layers)]
            x = torch.cat(x, 1)
            x = self.emb_dropout_layer(x)

        if self.no_of_cont != 0:
            normalized_cont_data = self.first_bn_layer(cont_data)

            if self.no_of_embs != 0:
                x = torch.cat([x, normalized_cont_data], 1) 
        else:
            x = normalized_cont_data

        for lin_layer, dropout_layer, bn_layer in\
            zip(self.lin_layers, self.droput_layers, self.bn_layers):

            x = F.relu(lin_layer(x))
            x = bn_layer(x)
            x = dropout_layer(x)

        x = self.output_layer(x)

        return x


In [102]:
from pathlib import Path
import pandas as pd
import numpy as np
# Using only a subset of the variables.
PATH = Path('../data/houseprice/')
data = pd.read_csv(PATH/"train.csv", usecols=["SalePrice", "MSSubClass", "MSZoning", "LotFrontage", "LotArea",
                                         "Street", "YearBuilt", "LotShape", "1stFlrSF", "2ndFlrSF"]).dropna()

In [103]:
data

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,YearBuilt,1stFlrSF,2ndFlrSF,SalePrice
0,60,RL,65.0,8450,Pave,Reg,2003,856,854,208500
1,20,RL,80.0,9600,Pave,Reg,1976,1262,0,181500
2,60,RL,68.0,11250,Pave,IR1,2001,920,866,223500
3,70,RL,60.0,9550,Pave,IR1,1915,961,756,140000
4,60,RL,84.0,14260,Pave,IR1,2000,1145,1053,250000
5,50,RL,85.0,14115,Pave,IR1,1993,796,566,143000
6,20,RL,75.0,10084,Pave,Reg,2004,1694,0,307000
8,50,RM,51.0,6120,Pave,Reg,1931,1022,752,129900
9,190,RL,50.0,7420,Pave,Reg,1939,1077,0,118000
10,20,RL,70.0,11200,Pave,Reg,1965,1040,0,129500


In [104]:
categorical_features = ["MSSubClass", "MSZoning", "Street", "LotShape", "YearBuilt"]
cont_features = ['LotFrontage','LotArea','1stFlrSF','2ndFlrSF']
output_feature = "SalePrice"

from sklearn.preprocessing import LabelEncoder
label_encoders = {}

for cat_col in categorical_features:
    label_encoders[cat_col] = LabelEncoder()
    data[cat_col] = label_encoders[cat_col].fit_transform(data[cat_col])


In [105]:
dataset = TabularDataset(data=data, cat_cols=categorical_features, output_col=output_feature)

In [106]:
batchsize = 64
dataloader = DataLoader(dataset, batchsize, shuffle=True, num_workers=1)

In [64]:
cat_dims = [int(data[col].nunique()) for col in categorical_features]
cat_dims

[15, 5, 2, 4, 112]

In [65]:
emb_dims = [(x, min(50, (x + 1) // 2)) for x in cat_dims]
emb_dims

[(15, 8), (5, 3), (2, 1), (4, 2), (112, 50)]

In [66]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FeedForwardNN(emb_dims, no_of_cont=4, lin_layer_sizes=[50, 100],
                          output_size=1, emb_dropout=0.04,
                          lin_layer_dropouts=[0.001,0.01]).to(device)

In [67]:
no_of_epochs = 5
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)


In [68]:
for epoch in range(no_of_epochs):
    for y, cont_x, cat_x in dataloader:
          
        cat_x = cat_x.to(device)
        cont_x = cont_x.to(device)
        y  = y.to(device)

        # Forward Pass
        preds = model(cont_x, cat_x)
        loss = criterion(preds, y)

        # Backward Pass and Optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


In [97]:
preds = model(cont_x, cat_x)

In [98]:
preds

tensor([[ 6484.6670],
        [32424.9453],
        [23160.4883],
        [15820.9590],
        [ 1487.1158],
        [ 7192.5640],
        [17669.5605],
        [ 3319.6470],
        [ 9736.7578],
        [39645.0195],
        [ 6388.4707],
        [ 8849.8164],
        [50807.1875],
        [ 3920.3667],
        [-1208.2036],
        [18605.2832],
        [ 6881.8667],
        [-6289.1309],
        [25750.9355],
        [ 8001.2314],
        [26300.0488],
        [25810.0605],
        [21370.6504],
        [ 6453.9336],
        [ 1972.7488],
        [50283.4805],
        [21040.7012],
        [27158.7441],
        [ -603.1664],
        [-2910.0503],
        [ 7855.2559],
        [ 5535.9927],
        [54284.1484],
        [30753.0410],
        [-1039.7599],
        [ 1334.4977],
        [32301.1895],
        [ 7279.9736],
        [ 7233.2334],
        [10097.1133],
        [ 3747.5852],
        [30234.8496],
        [  263.3323],
        [ 3527.8069],
        [52318.5859],
        [1

In [83]:
df_cat = torch.tensor(data[categorical_features].values, dtype=torch.long, device=device)
df_cont = torch.tensor(data[cont_features].values, dtype=torch.float32, device=device)
preds = model(df_cont, df_cat)

In [88]:
df_cat.shape

torch.Size([1201, 5])